# Python第十二章学习

## 进程与线程

- [x] 多进程
- [x] 多线程

## 进程与线程
- 什么是进程呢？对于操作系统而言，一个任务就是一个进程，当我们打开浏览器时就启动了一个浏览器进程，打开两个就启动了两个进程

- 什么是线程？有些进程不只是同时干一件事，比如word，我们可以同时打字，拼写检查，打印等，在一个进程内部要干多件事情，这些子任务就是线程（Thread）

- 一个进程至少一个线程，多任务的实现有3种方式：
    - 多进程程模式（一对一）
    - 多线程模式（一对多）
    - 多进程+多线程模式（多对多）
    
首先我们先来看一下多进程：

### 多进程
- 现在我们是要让Python程序实现多进程（multiprocessing）

- Unix/Linux操作系统提供了一个fork()系统调用，它很特殊，不同的函数调用，调用一次，返回一次，但是fork()调用一次返回两次，因为操作系统自动把当前进程（父进程）复制了一份（子进程），然后，分别在父进程和子进程内返回

- 子进程永远返回0，而父进程返回子进程的id，这样做，一个父进程可以fork出很多子进程，所以，父进程要记下每个子进程的id，而子进程只需要getppid()就可以拿到父进程的ID

如果打算编写多进程的服务程序，Unix/Linux无疑是正确的选择，由于windows没有fork调用，所以Python提供了一个multiprocessing模块就是跨平台的多进程模块。

multiprocessing模块提供了一个Process类来代表一个进程对象

In [16]:
from multiprocessing import Process
import os

def run_proc(name):
    print("Run child process %s (%s)..." % (name,os.getpid()))
    
if __name__=="__main__":
    print("Parent pricess %s." % os.getpid())
    p = Process(target = run_proc,args= ("test",))
    print("Child process will start.")
    p.start()
    p.join()
    print("Child process end.")

Parent pricess 38700.
Child process will start.
Run child process test (38700)...
Child process end.


In [13]:
#help(Process)

创建子进程时，只需要传入一个执行函数和函数的参数，创建一个Process实例，用start()方法启动，这样创建进程比fork()还要简单。

join()方法可以等待子进程结束后再继续往下运行，通常用于进程间的同步。

#### 子进程
- 很多时候，子进程并不是自身，而是一个外部进程。我们创建了子进程后，还需要控制子进程的输入和输出。

- subprocess模块可以让我们非常方便地启动一个子进程，然后控制其输入和输出。

- 下面的例子演示了如何在Python代码中运行命令`nslook upwww.python.org`，这和命令行直接运行的效果是一样的：

In [17]:
import subprocess

print('$ nslookup www.python.org')
r = subprocess.call(['nslookup', 'www.python.org'])
print('Exit code:', r)

$ nslookup www.python.org
Exit code: 0


如果子进程还需要输入，则可以通过communicate()方法输入：

In [19]:
import subprocess

print('$ nslookup')
p = subprocess.Popen(['nslookup'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
output, err = p.communicate(b'set q=mx\npython.org\nexit\n')
print(output.decode('gbk'))
print('Exit code:', p.returncode)

$ nslookup
默认服务器:  center.shao.ac.cn
Address:  202.127.29.4

> > 服务器:  center.shao.ac.cn
Address:  202.127.29.4

python.org	MX preference = 50, mail exchanger = mail.python.org

python.org	nameserver = ns1.p11.dynect.net
python.org	nameserver = ns3.p11.dynect.net
python.org	nameserver = ns2.p11.dynect.net
python.org	nameserver = ns4.p11.dynect.net
mail.python.org	internet address = 188.166.95.178
mail.python.org	AAAA IPv6 address = 2a03:b0c0:2:d0::71:1
ns2.p11.dynect.net	internet address = 204.13.250.11
ns4.p11.dynect.net	internet address = 204.13.251.11
ns1.p11.dynect.net	internet address = 208.78.70.11
ns3.p11.dynect.net	internet address = 208.78.71.11
> 
Exit code: 0


#### 进程间通信
- Process之间肯定是需要通信的，操作系统提供了很多机制来实现进程间的通信。Python的multiprocessing模块包装了底层的机制，提供了Queue、Pipes等多种方式来交换数据。

- 我们以Queue为例，在父进程中创建两个子进程，一个往Queue里写数据，一个从Queue里读数据：

In [24]:
from multiprocessing import Process, Queue
import os, time, random
import pickle

# 写数据进程执行的代码:
def write(q):
    print('Process to write: %s' % os.getpid())
    for value in ['A', 'B', 'C']:
        print('Put %s to queue...' % value)
        q.put(value)
        time.sleep(random.random())

# 读数据进程执行的代码:
def read(q):
    print('Process to read: %s' % os.getpid())
    while True:
        value = q.get(True)
        print('Get %s from queue.' % value)

if __name__=='__main__':
    # 父进程创建Queue，并传给各个子进程：
    q = Queue()
    pw = Process(target=write(q))
    pr = Process(target=read, args=(q,))
    # 启动子进程pw，写入:
    pw.start()
    # 启动子进程pr，读取:
    pr.start()
    # 等待pw结束:
    pw.join()
    # pr进程里是死循环，无法等待其结束，只能强行终止:
    pr.terminate()

Process to write: 38700
Put A to queue...
Put B to queue...
Put C to queue...


### 多线程
- 多任务可以有多进程完成，也可以由一个进程内的多线程完成
- 由于线程是操作系统直接支持的执行单元，因此，高级语言通常都内置多线程的支持，Python也不例外，并且，python的线程是真正的Posix Thread，而不是模拟出来的线程

- Python的标准库提供了两个模块：\_thread和threading，\_thread是低级模块，threading是高级模块，对\_thread进行了封装，绝大多数哦情况下，我们只需要使用threading这个高级模块

- 启动一个线程就是把一个函数并创建Thread实例，然后调用start()开始执行：

In [36]:
import time,threading

def loop():
    print("thread %s is running..." % threading.current_thread().name )
    n = 0
    while n < 5:
        n = n + 1
        print("thread %s >>> %s" % (threading.current_thread().name, n))
        time.sleep(0.1)
    print("thread %s ended" % threading.current_thread().name)

print("thread %s is running..." % threading.current_thread().name)
t = threading.Thread(target=loop, name="LoopThread")
t.start()
t.join()
print("thread %s ended." % threading.current_thread().name)

thread MainThread is running...
thread LoopThread is running...
thread LoopThread >>> 1
thread LoopThread >>> 2
thread LoopThread >>> 3
thread LoopThread >>> 4
thread LoopThread >>> 5
thread LoopThread ended
thread MainThread ended.


由于任何进程默认就会启动一个线程，我们把该线程称为主线程，主线程又可以启动新的线程，Python的threading模块有个current_thread()函数，它永远返回当前线程的实例。主线程实例的名字叫MainThread，子线程的名字在创建时指定，我们用LoopThread命名子线程。名字仅仅在打印时用来显示，完全没有其他意义，如果不起名字Python就自动给线程命名为Thread-1，Thread-2……

#### lock
- 多线程和多进程最大的不同在于，在多进程中，同一个变量，各自有一份拷贝存在于每个进程中，互不影响，在多线程中，所有变量都由所有进程共享，所以任何一个变量都可以被任何一个线程修改，因此，线程之间共享数据最大的危险在于多个线程同时改一个变量，把内容给改乱了。


In [ ]:
import time,threading

balance = 0

def change_it(n):
    global balance
    balance = balance + n
    balance = balance - n
    
def run_thread(n):
    for i in range(100000):
        change_it(n)
        
t1 = threading.Thread(target = run_thread, args = (5,))
t2 = threading.Thread(target = run_thread, args = (8,))
t1.start()
t2.start()
t1.join()
t2.join()
print(balance)

我们定义了一个共享变量balance，初始值为0，并且启动两个线程，先存后取，理论上结果应该为0，但是，由于线程的调度是由操作系统决定的，当t1、t2交替执行时，只要循环次数足够多，balance的结果就不一定是0了。

原因是因为高级语言的一条语句在CPU执行时是若干条语句，即使一个简单的计算：
```
balance = balance + n
```
也分两步：
1. 计算balance + n ，存入临时变量中；
2. 将临时变量的值赋给balance。

也就是可以看成：
```
x = balance + n
balance = x
```
由于x是局部变量，两个线程各自都有自己的x，当代码正常执行时：


初始值balance = 0
```
t1：x1 = balance + 5
t1: balance = x1
t1: x1 = balance - 5
t1: balance = x1

t2: x2 = balance + 8
t2：balance = x2
t2: x2 = balance - 8
t2: balance = x2
```

结果balance = 0 


但是t1和t2是交替运行的，如果操作系统以下面的顺序执行t1、t2

初始值 balance = 0
```
t1: x1 = balance + 5  

t2: x2 = balance + 8  
t2: balance = x2      

t1: balance = x1     
t1: x1 = balance - 5  
t1: balance = x1      

t2: x2 = balance - 8  
t2: balance = x2   
```
结果 balance = -8

究其原因，是因为修改balance需要多条语句，而执行这几条语句时，线程可能中断，从而导致多个线程把同一个对象的内容改乱了。

两个线程同时一存一取，就可能导致余额不对，你肯定不希望你的银行存款莫名其妙地变成了负数，所以，我们必须确保一个线程在修改balance的时候，别的线程一定不能改。

如果我们要确保balance计算正确，就要给change_it()上一把锁，当某个线程开始执行change_it()时，我们说，该线程因为获得了锁，因此其他线程不能同时执行change_it()，只能等待，直到锁被释放后，获得该锁以后才能改。由于锁只有一个，无论多少线程，同一时刻最多只有一个线程持有该锁，所以，不会造成修改的冲突。创建一个锁就是通过threading.Lock()来实现：

In [2]:
import time,threading

balance = 0
lock = threading.Lock()

def run_thread(n):
    for i in range(100000):
        # 先要获取锁:
        lock.acquire()
        try:
            # 放心地改吧:
            change_it(n)
        finally:
            # 改完了一定要释放锁:
            lock.release()

当多个线程同时执行lock.acquire()时，只有一个线程能成功的获取锁，然后继续执行代码，其他线程继续等待直到获取锁为止。

获得锁的线程用完后一定要释放锁，否则其他进行就会一直处于等待状态，称为死线程，所以我们用try...finally来确保锁一定会被释放

锁的好处就是确保了某段关键代码只能由一个线程从头到尾完整的执行，坏处当然也很多，首先是阻止了多进程并发执行，包含锁的某段代码实际上只能以但线程模式执行，效率就就降低很多，其次，由于可以存在多个锁，不同的线程有不同的锁，并试图获取对方持有的锁时，可能会造成死锁，导致对个线程全部挂起，既不能执行，也无法结束，只能靠操作系统强行终止